# Abgabe 3 Mark Nagengast Porro, Tim Schwabe

In [24]:
import numpy as np

## Aufgabe 1

In [25]:
def RELU(x):
    return max(0,x)

In [26]:
def sigmoid(x):
    return 1/(1+(np.math.e)**(-x))

In [27]:
def createTrainingData(batch_size):

    data = np.array([[0,0],
                     [0,1],
                     [1,0],
                     [1,1]])
    truth = np.array([[0],[1],[1],[0]])
    
    training_data = []
    target = []
    while(batch_size > 0):
        index = np.random.randint(4)
        training_data.append(data[index])
        target.append(truth[index])
        batch_size -= 1
    return np.array(training_data), np.array(target)

train, target = createTrainingData(10)
#print(train)
#print(target)

## Aufgabe 2

In [57]:
class NN():

    def __init__(self, training_data, target_data,hidden_layers):

        self.training_data = training_data
        self.target_data = target_data

        self.weights = [np.random.uniform(size=(2,4))]
        while(hidden_layers > 1):
            self.weights.append(np.random.uniform(size=(4,4)))
            hidden_layers -=1
        self.weights.append(np.random.uniform(size=(1,4)))
        print(self.weights)


    def ReLU(self,x):
        return x * (x > 0) 
    
    def sigmoid(self, x):
        return 1/(1+(np.math.e)**(-np.array(x)))
    
    def forward(self, x):
        for i in range(len(self.weights)-1):
            product = np.dot(x, self.weights[i])
            x = self.ReLU(np.array(product))
        return self.sigmoid(np.dot(self.weights[-1], x))
    
    def execute(self,x):
        return np.round(self.forward(x)[0])

    def training(self):

        for i in range(len(self.training_data)):
            print(self.training_data[i])
            print(self.forward(self.training_data[i]))
        

In [58]:
test = NN(train,target,2)
test.execute([0,1])

[array([[0.1387762 , 0.4702463 , 0.13235123, 0.81664225],
       [0.32150941, 0.60263201, 0.36966742, 0.13933313]]), array([[0.66117203, 0.14140535, 0.07667112, 0.3288914 ],
       [0.01516861, 0.80969122, 0.67609459, 0.422725  ],
       [0.68156576, 0.67079619, 0.13764437, 0.06916524],
       [0.77952527, 0.17529651, 0.59602891, 0.88679663]]), array([[0.99565537, 0.10105116, 0.46204335, 0.42219197]])]


1.0